In [1]:
import pandas as pd
from pycoingecko import CoinGeckoAPI
import time
from datetime import date
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from turtle import width
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import dash_table
import plotly.io as pio

pio.renderers.default = 'notebook_connected'

/var/folders/mw/_f0gy0f54t739qb79y8cxb_40000gn/T/ipykernel_17302/100811200.py:12: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
/var/folders/mw/_f0gy0f54t739qb79y8cxb_40000gn/T/ipykernel_17302/100811200.py:13: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
/var/folders/mw/_f0gy0f54t739qb79y8cxb_40000gn/T/ipykernel_17302/100811200.py:15: UserWarning: 
The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`
  import dash_table


In [2]:
etherscan_transaction_csv = "/Users/eamonmcandrew/Downloads/export-0x3598f618d45e02b92d9b4d8129d45dbd1b0beb5d_Jan_21-Nov_21.csv"



In [3]:
class strong_etherscan_parser:

    def __init__(self, etherscan_trasasction_csv):
        # read csv
        self.etherscan_trasasction_csv = etherscan_trasasction_csv
        
        # create a pandas dataframe
        self.etherscan_transaction_df = pd.read_csv(self.etherscan_trasasction_csv, index_col=False)
        
        
    @property
    def strong_claims_dataframe(self):
        dataframe = self.etherscan_transaction_df
        claim_df = dataframe[dataframe["Method"] == "Claim All"]
        claim_df["Value_out_USD"] = claim_df["Value_OUT(ETH)"] * claim_df["Historical $Price/Eth"]
        final_info_claim_df = claim_df[["Txhash", "DateTime", "TxnFee(ETH)", "Value_out_USD", "Value_OUT(ETH)", "TxnFee(USD)"]]
        final_info_claim_df["Fee_eth_total"] = final_info_claim_df["Value_OUT(ETH)"] + final_info_claim_df["TxnFee(ETH)"]
        return final_info_claim_df
    
    
    @property
    def get_price_to_date(currency="strong", start_date="01/01/2021"):
        cg = CoinGeckoAPI()
        cg.get_price(ids='strong', vs_currencies='Eur')

        start_date = start_date
        start_date = pd.to_datetime(start_date)
        today = date.today()

        date_range = [start_date, today]
        date_range = list(map(pd.to_datetime, date_range))
        date_range = list(map(lambda x:x.strftime('%s'), date_range))
        chart_data_from_API = cg.get_coin_market_chart_range_by_id(id='strong',vs_currency='eur',from_timestamp=date_range[0],to_timestamp=date_range[1])

        df_list = []
        for key in chart_data_from_API.keys():
            df = pd.DataFrame.from_dict(chart_data_from_API[key])
            df = df.rename(columns={0: "Date", 1: key})
            df["Date"] = pd.to_datetime(df["Date"], unit='ms')
            # df["Date"] = pd.to_datetime(df["Date"], format='%Y%m%d%H%M%S', errors='coerce')
            df= df.set_index("Date")
            df_list.append(df)
        price_df = pd.concat(df_list, axis=1)
        price_df = price_df.rename(columns={"prices": "price"})

        return price_df

In [4]:
transaction_object = strong_etherscan_parser(etherscan_transaction_csv)

In [5]:
final_claim_info_df = transaction_object.strong_claims_dataframe

/var/folders/mw/_f0gy0f54t739qb79y8cxb_40000gn/T/ipykernel_17302/2804270688.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/mw/_f0gy0f54t739qb79y8cxb_40000gn/T/ipykernel_17302/2804270688.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [6]:
final_claim_info_df[["DateTime", "Fee_eth_total"]]

final_claim_info_df["DateTime"] = pd.to_datetime(final_claim_info_df["DateTime"])

In [7]:
final_claim_info_df.DateTime

50    2021-02-23 18:35:40
58    2021-02-28 08:41:40
66    2021-03-08 12:54:35
103   2021-04-06 11:47:23
116   2021-04-15 12:01:55
121   2021-04-24 10:45:36
125   2021-05-15 19:40:27
130   2021-06-03 09:55:36
134   2021-06-10 10:38:10
136   2021-06-18 10:33:35
138   2021-06-24 09:25:44
141   2021-07-02 11:35:16
144   2021-07-08 09:41:39
146   2021-07-15 09:39:39
152   2021-07-30 09:20:32
159   2021-08-16 08:16:15
185   2021-09-19 13:22:17
218   2021-10-23 08:37:05
229   2021-11-05 09:49:22
254   2021-11-13 09:33:48
261   2021-11-23 12:18:53
263   2021-11-28 10:50:41
Name: DateTime, dtype: datetime64[ns]

transaction_object.get_price_to_date[["Date", "price"]]

In [8]:
price = transaction_object.get_price_to_date["price"]

In [9]:
print(price.index.min())
print(price.index.max())
print(date.today())

2021-01-01 00:00:00
2022-08-24 00:00:00
2022-08-25


In [10]:
final_claim_info_df["Date"] = pd.to_datetime(final_claim_info_df["DateTime"]).dt.date

final_claim_info_df.set_index('Date', inplace=True)

In [11]:
final_claim_info_df

,Txhash,DateTime,TxnFee(ETH),Value_out_USD,Value_OUT(ETH),TxnFee(USD),Fee_eth_total
Date,,,,,,,
2021-02-23,0x5b89779d7be888a7f98a3ad4088ed20e18312c645455...,2021-02-23 18:35:40,0.094760,16.553055,0.010489,156.648298,0.105249
2021-02-28,0x7e393d5e43960134d271199d14114e423ad34b49e05f...,2021-02-28 08:41:40,0.065796,5.186572,0.003646,108.768012,0.069442
2021-03-08,0x5cf5155a0962e39b902085a19f038baa3ff1986152bf...,2021-03-08 12:54:35,0.075236,10.541883,0.005747,124.373412,0.080983
2021-04-06,0x845afa670fad27c16a39aa81688f61ffef54de3ffc43...,2021-04-06 11:47:23,0.128674,49.607534,0.023485,212.711687,0.152160
2021-04-15,0x46f70605a97c4051c814cc9ace91f35199ce0a16dc57...,2021-04-15 12:01:55,0.064522,20.518516,0.008152,106.662145,0.072674
2021-04-24,0x51c474efacc2de2305eef281e504d700362fc1480440...,2021-04-24 10:45:36,0.065830,12.080126,0.005450,108.822795,0.071279
2021-05-15,0x23053c754685f7d23047fbdbf2d17e156671e3ac82ab...,2021-05-15 19:40:27,0.080982,46.166229,0.012669,133.871017,0.093651
2021-06-03,0x1f78c80b67e20a9b1a723af75a252b772ea9bf671bb1...,2021-06-03 09:55:36,0.024295,31.397469,0.010993,40.161305,0.035288
2021-06-10,0x7d43311a1bcc920f226ad0e4dfe0e8488c9ad28d3190...,2021-06-10 10:38:10,0.020377,13.629338,0.005514,33.685146,0.025891


In [12]:
merged = pd.merge(price,final_claim_info_df, how='right', left_index=True, right_index=True)

In [13]:
merged

,price,Txhash,DateTime,TxnFee(ETH),Value_out_USD,Value_OUT(ETH),TxnFee(USD),Fee_eth_total
Date,,,,,,,,
2021-02-23,59.323296,0x5b89779d7be888a7f98a3ad4088ed20e18312c645455...,2021-02-23 18:35:40,0.094760,16.553055,0.010489,156.648298,0.105249
2021-02-28,43.165026,0x7e393d5e43960134d271199d14114e423ad34b49e05f...,2021-02-28 08:41:40,0.065796,5.186572,0.003646,108.768012,0.069442
2021-03-08,52.364445,0x5cf5155a0962e39b902085a19f038baa3ff1986152bf...,2021-03-08 12:54:35,0.075236,10.541883,0.005747,124.373412,0.080983
2021-04-06,146.393856,0x845afa670fad27c16a39aa81688f61ffef54de3ffc43...,2021-04-06 11:47:23,0.128674,49.607534,0.023485,212.711687,0.152160
2021-04-15,151.666811,0x46f70605a97c4051c814cc9ace91f35199ce0a16dc57...,2021-04-15 12:01:55,0.064522,20.518516,0.008152,106.662145,0.072674
2021-04-24,122.128605,0x51c474efacc2de2305eef281e504d700362fc1480440...,2021-04-24 10:45:36,0.065830,12.080126,0.005450,108.822795,0.071279
2021-05-15,136.860669,0x23053c754685f7d23047fbdbf2d17e156671e3ac82ab...,2021-05-15 19:40:27,0.080982,46.166229,0.012669,133.871017,0.093651
2021-06-03,109.250218,0x1f78c80b67e20a9b1a723af75a252b772ea9bf671bb1...,2021-06-03 09:55:36,0.024295,31.397469,0.010993,40.161305,0.035288
2021-06-10,112.956758,0x7d43311a1bcc920f226ad0e4dfe0e8488c9ad28d3190...,2021-06-10 10:38:10,0.020377,13.629338,0.005514,33.685146,0.025891


In [14]:
fig = go.Figure([go.Scatter(x=price.index, y=price.values, line_color="grey",  name = "Strong Price")])

fig.add_traces(go.Scatter(x=merged.index, y=merged['price'],name = "Rewards Claimed",
                          textposition='top left',
                          textfont=dict(color='#233a77'),
                          opacity=0.6,
                          mode='markers+text',
                          marker=dict(color='#19D3F3', size=8),
                        #   text = Node_creation_df["tgl"])
                        )),

# fig.add_traces(go.Scatter(x=Node_creation_df.index, y=Node_creation_df['price'],name = "Node Creation",
#                           textposition='top left',
#                           textfont=dict(color='#233a77'),
#                           opacity=0.6,
#                           mode='markers+text',
#                           marker=dict(color='#AB63FA', size=8),
#                         #   text = Node_creation_df["tgl"])
#                         ))

fig.update_layout(xaxis_title='Date', yaxis_title='Price in ')
fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)

fig.show()

In [19]:
from etherscan import Etherscan

api_key = "DTB4IIBEJAE8IWV9NJ8YA7AA5FHEKFI62N"
eth = Etherscan(api_key)


In [ ]:
wallet_address = "x0x3598F618d45e02B92d9b4D8129d45DbD1b0beB5D"